In [1]:
import os
import pandas as pd
import numpy as np
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D
#from tensorflow.python.keras.preprocessing.image import ImageDataGenerator, load_img
import matplotlib.pyplot as plt
import cv2
import warnings
import tensorflow as tf
from keras.models import Sequential
from keras.optimizers import Adam
tf.random.set_seed(42)
#warnings.filterwarnings("ignore")

In [2]:
# If this is not set, bug because tf takes too much memory
physical_devices = tf.config.experimental.list_physical_devices('GPU')
assert len(physical_devices) > 0, "Not enough GPU hardware devices available"
config = tf.config.experimental.set_memory_growth(physical_devices[0], True)

Init variables (in the script for now, via config file later)

In [3]:
config = None

# Global
index = 0

# For data
path = '/home/mlmaster/Code/Ing_ml_P6/dataset/'
img_height = 80
img_width = 80
zca_whitening = False
horizontal_flip = True

# For model
model_type = 3
batch_size = 50
learning_rate = 0.001
epochs = 50
steps_per_epoch = int(12000 / (batch_size))
validation_steps = int(1200 / (batch_size))

# For Logs
save = "/home/mlmaster/Code/Ing_ml_P6/logs/"

Building Dataset for logs

In [4]:
dataset = [index, img_height, img_width, batch_size,
           learning_rate, epochs, steps_per_epoch,
           validation_steps, zca_whitening, horizontal_flip,
           model_type]

dataset = pd.DataFrame(dataset).T
dataset = dataset.rename(columns={0:'Index', 1:'Img Height', 2:'Img Width', 3:'Batch Size',
                                  4:'Learning Rate', 5:'Epochs', 6:'Steps Per Epoch',
                                  7:'validation Steps', 8:'Zca Whitening', 9:'Horizontal Flip',
                                  10:'Model Type'})

In [5]:
dataset

,Index,Img Height,Img Width,Batch Size,Learning Rate,Epochs,Steps Per Epoch,validation Steps,Zca Whitening,Horizontal Flip,Model Type
0,0,80,80,50,0.001,50,240,24,False,True,3


In [6]:
# Locked variable, does not change with config file
train = 'Train/'
val = 'Validation/'
test = 'Test/'
fullPathTrain = path + train
fullPathVal = path + val
fullPathTest = path + test

# For model
train_datagen =  ImageDataGenerator(
    zca_whitening=zca_whitening,
    horizontal_flip=horizontal_flip,
    rescale=1./255
)

val_datagen =  ImageDataGenerator(
    rescale=1./255
)

test_datagen =  ImageDataGenerator(
    rescale=1./255
)

In [7]:
#setting the path to the directory containing the pics
categories = []
for i in os.listdir(fullPathTrain):
    categories.append(i)

In [8]:
def load_data(which='Train', categories=categories):
    
    """Function that loads data and resize the images"""
    
    if which == 'Train':
        path = fullPathTrain
        print('Loading Train Data')
    elif which == 'Val':
        path = fullPathVal
        print('Loading Validation Data')
    elif which == 'Test':
        path = fullPathTest
        print('Loading Test Data')
    else:
        print('error, which must be either : Train, Val, Test')
    data = []
    label = []
    for i in categories:
        for img in os.listdir(path + i):
            pic = cv2.imread(os.path.join(path + i,img))
            pic = cv2.cvtColor(pic,cv2.COLOR_BGR2RGB)
            pic = cv2.resize(pic,(img_height, img_width))
            data.append([pic])
            label.append(i)
    print(len(data), " Images loaded")
    # Formating data
    data = np.array(data)
    data = data.reshape((data.shape[0], img_height, img_width, 3))
    data = data.astype('float32')
    label = np.array(label)
    label = [i.split('-')[1] for i in label]
    
    return data, label

Fit if zca_whitening is True

In [9]:
if zca_whitening == True:
    dataToFit, label = load_data(which='Train', categories=categories)
    dataToFit = dataToFit.reshape((dataToFit.shape[0], img_height, img_width, 3))
    dataToFit = dataToFit.astype('float32')
    train_datagen.fit(dataToFit)
    # If set to True, uses the CPU instead of GPU to train the model ; See with Denis why

In [10]:
# Defining the generator
print('Total number of images for "training":')
train_generator = train_datagen.flow_from_directory(
                                fullPathTrain,
                                target_size = (img_height, img_width),
                                batch_size = batch_size, 
                                class_mode = "categorical")

print('Total number of images for "validation":')
val_generator = test_datagen.flow_from_directory(
                             fullPathVal,
                             target_size = (img_height, img_width),
                             batch_size = batch_size,
                             class_mode = "categorical",
                             shuffle=False)

print('Total number of images for "testing":')
test_generator = test_datagen.flow_from_directory(
                              fullPathTest,
                              target_size = (img_height, img_width),
                              batch_size = batch_size,
                              class_mode = "categorical",
                              shuffle=False)

Total number of images for "training":
Found 12348 images belonging to 120 classes.
Total number of images for "validation":
Found 4116 images belonging to 120 classes.
Total number of images for "testing":
Found 4116 images belonging to 120 classes.


In [11]:
# Model Builder
if model_type == 1:
    model = Sequential()
    model.add(Conv2D(64, kernel_size=3, activation='relu', input_shape=(img_height,img_width,3)))
    model.add(Conv2D(32, kernel_size=3, activation='relu', input_shape=(img_height,img_width,3)))
    model.add(Conv2D(16, kernel_size=3, activation='relu', input_shape=(img_height,img_width,3)))
    model.add(Conv2D(8, kernel_size=3, activation='relu'))
    model.add(Flatten())
    model.add(Dense(len(categories), activation='softmax'))
    
elif model_type == 2:
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(5, 5), strides=(1, 1),
                     activation='relu',
                     input_shape=(img_height,img_width,3)))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    
    model.add(Conv2D(64, (5, 5), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    
    model.add(Dense(1000, activation='relu'))
    model.add(Dense(len(categories), activation='softmax'))
    
elif model_type == 3:
    model = Sequential()
    model.add(Conv2D(8, kernel_size=(5, 5), strides=(1, 1),
                     activation='relu',
                     input_shape=(img_height,img_width,3)))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    
    model.add(Conv2D(16, (5, 5), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    model.add(Conv2D(32, kernel_size=(5, 5), strides=(1, 1),
                     activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    
    model.add(Conv2D(64, (5, 5), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    
    model.add(Dense(1000, activation='relu'))
    model.add(Dense(len(categories), activation='softmax'))
    
elif model_type == 4:
    model = Sequential()
    model.add(Conv2D(64, kernel_size=(5, 5), strides=(2, 2),
                     activation='relu',
                     input_shape=(img_height,img_width,3)))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(1, 1)))
    
    model.add(Conv2D(32, (5, 5), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    
    model.add(Dense(1000, activation='relu'))
    model.add(Dense(len(categories), activation='softmax'))    

In [12]:
# Optmizer and compilation
opt = Adam(lr=learning_rate, clipnorm=1.)
model.compile(loss='binary_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 76, 76, 8)         608       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 38, 38, 8)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 34, 34, 16)        3216      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 17, 17, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 13, 13, 32)        12832     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 6, 6, 32)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 2, 2, 64)          5

In [14]:
# Model fit via generator
history = model.fit(train_generator,
                    epochs=epochs,
                    validation_data=val_generator,
                    steps_per_epoch=steps_per_epoch,
                    validation_steps=validation_steps,
                    verbose=1)

Epoch 1/50
240/240 [==============================] - 26s 110ms/step - loss: 0.0476 - accuracy: 0.0142 - val_loss: 0.0469 - val_accuracy: 0.0317
Epoch 2/50
240/240 [==============================] - 26s 110ms/step - loss: 0.0453 - accuracy: 0.0363 - val_loss: 0.0445 - val_accuracy: 0.0433
Epoch 3/50
240/240 [==============================] - 26s 110ms/step - loss: 0.0441 - accuracy: 0.0479 - val_loss: 0.0429 - val_accuracy: 0.0500
Epoch 4/50
240/240 [==============================] - 26s 109ms/step - loss: 0.0432 - accuracy: 0.0587 - val_loss: 0.0437 - val_accuracy: 0.0429
Epoch 5/50
240/240 [==============================] - 26s 110ms/step - loss: 0.0424 - accuracy: 0.0698 - val_loss: 0.0430 - val_accuracy: 0.0542
Epoch 6/50
240/240 [==============================] - 27s 111ms/step - loss: 0.0417 - accuracy: 0.0776 - val_loss: 0.0426 - val_accuracy: 0.0725
Epoch 7/50
240/240 [==============================] - 26s 108ms/step - loss: 0.0411 - accuracy: 0.0871 - val_loss: 0.0418 - val_ac

KeyboardInterrupt: 

Ploting graph

In [ ]:
"""path_img = save + str(index) + '/'
try:
    os.mkdir(path_img)
except:
    print('This directory already exists, please make sure that you changed the index')"""

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1,len(acc)+1)

plt.figure()
plt.plot(epochs, acc, 'b', label = 'Training accuracy')
plt.plot(epochs, val_acc, 'r', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()
#plt.savefig(path_img + 'Accuracy.jpg')
plt.plot()

plt.figure()
plt.plot(epochs, loss, 'b', label = 'Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
#plt.savefig(path_img + 'Loss.jpg')
plt.plot()

Saving best scores

In [ ]:
val_acc = max(history.history['val_accuracy'])
val_loss = max(history.history['val_loss'])

dataset['Validation Accuracy'] = val_acc
dataset['Validation Loss'] = val_loss

In [ ]:
"""try:
    logs = pd.read_csv(save + 'logs.csv')
    logs = pd.concat([logs, dataset])
    logs.to_csv(save + 'logs.csv')
except:
    dataset.to_csv(save + 'logs.csv')"""

In [ ]:
"""for i in dataset['Index']:
    print(dataset.loc[0, 'Validation Accuracy'])"""

In [ ]:
import keras
from numba import cuda
# Clearing session as we will train model many time
keras.backend.clear_session()
# Releasing gpu memory directly with cuda
cuda.select_device(0)
cuda.close()